In [20]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

Loading the Data

In [21]:
# Download training data from open datasets.
training_data = datasets.CIFAR10(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.CIFAR10(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

Files already downloaded and verified
Files already downloaded and verified


Creating Iterable dataloaders to facilitate creating batches

In [22]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 3, 32, 32])
Shape of y: torch.Size([64]) torch.int64


Defining the model

In [23]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(32*32*3, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=3072, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Declaring loss function and optimizer

In [24]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

Training the model

In [25]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

Testing the model

In [26]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

Running the model and testing after each epoch

In [27]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.312232  [   64/50000]
loss: 2.304405  [ 6464/50000]
loss: 2.278688  [12864/50000]
loss: 2.294687  [19264/50000]
loss: 2.275016  [25664/50000]
loss: 2.277276  [32064/50000]
loss: 2.278505  [38464/50000]
loss: 2.277255  [44864/50000]
Test Error: 
 Accuracy: 13.9%, Avg loss: 2.265833 

Epoch 2
-------------------------------
loss: 2.289031  [   64/50000]
loss: 2.280937  [ 6464/50000]
loss: 2.232387  [12864/50000]
loss: 2.268848  [19264/50000]
loss: 2.237917  [25664/50000]
loss: 2.240592  [32064/50000]
loss: 2.251446  [38464/50000]
loss: 2.237175  [44864/50000]
Test Error: 
 Accuracy: 19.3%, Avg loss: 2.226482 

Epoch 3
-------------------------------
loss: 2.259111  [   64/50000]
loss: 2.245825  [ 6464/50000]
loss: 2.173705  [12864/50000]
loss: 2.235104  [19264/50000]
loss: 2.188153  [25664/50000]
loss: 2.190585  [32064/50000]
loss: 2.218746  [38464/50000]
loss: 2.180613  [44864/50000]
Test Error: 
 Accuracy: 23.0%, Avg loss: 2.173136 

Epoc

Saving the model

In [28]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


Build the confusion matrix

In [51]:
import plotly.express as px

# constant for classes
classes = ("airplanes", "cars", "birds", "cats", "deer", "dogs", "frogs", "horses", "ships", "trucks")

zf = [[0 for pclass in classes] for aclass in classes]
zm = [0 for aclass in classes]

# iterate over test data
size = len(test_dataloader.dataset)
num_batches = len(test_dataloader)
model.eval()
test_loss, correct = 0, 0
with torch.no_grad():
    for X, y in test_dataloader:
        X, y = X.to(device), y.to(device)
        for Xi, yi in zip(X, y):
            #print(Xi.unsqueeze(0).shape)
            pred = model(Xi.unsqueeze(0))
            pi = pred.cpu().numpy().argmax()
            ai = yi.cpu().numpy()
            # print(pred.cpu().numpy().argmax())
            # print(yi.cpu().numpy())
            zf[ai][pi] += 1
            zm[ai] += 1
        #pred = model(X)
#         test_loss += loss_fn(pred, y).item()
#         correct += (pred.argmax(1) == y).type(torch.float).sum().item()
# test_loss /= num_batches
# correct /= size

# print(z)

zp = [[zf[ai][pi] / zm[ai]  for pi in range(10)] for ai in range(10)]

labels = ("airplanes", "cars", "birds", "cats", "deer", "dogs", "frogs", "horses", "ships", "trucks")

fig = px.imshow(zp, x=labels, y=labels, text_auto=True)
fig.show()